# pyQuil - t|ket> Retargetability Example

Rigetti's pyQuil package provides a clear and well-documented syntax for circuit construction, along with access to the Quil compiler, QVM simulator, and Quantum Cloud Services for access to real hardware devices. It is also the standard output for circuits from Grove.

t|ket> provides users with the luxury of choice for the combination of front-end (high-level algorithms and circuit construction) and back-end (device/simulator). This tutorial will demonstrate this by taking a pyQuil program generated by Grove and running it on Qiskit's Aer simulator, applying some optimisations in the middle.

Grove provides a nifty tool for arbitrary state generation - given a desired state, it constructs a circuit which maps the ground state to this target. However, it is not immediately obvious how this would act on other initial states. Qiskit's Aer provides a unitary simulator, giving the full unitary matrix for a given circuit which would be useful for this. We will use pytket as an intermediate step to connect the two together.

Let's start by setting up the arbitrary state preparation:

In [1]:
from grove.alpha.arbitrary_state.arbitrary_state import create_arbitrary_state
import numpy as np
target_state = np.array([0.5, 1j*0.5, -1j*0.5, 1j*0.5])
quil_circ = create_arbitrary_state(target_state)
print(quil_circ)

RZ(-pi/4) 0
PHASE(pi/4) 0
H 0
H 1
RZ(-pi/4) 0
SWAP 0 1
CNOT 1 0
CNOT 1 0
CNOT 1 0
RZ(-pi/4) 0
CNOT 1 0
RZ(3*pi/4) 0



The next step is to use pytket to convert this to a t|ket> circuit:

In [2]:
from pytket.pyquil import pyquil_to_tk
tk_circ = pyquil_to_tk(quil_circ)

Whilst we are here, let's apply some optimisations. Both `optimise_pre_routing` and `optimise_post_routing` convert the circuit into a Qiskit-friendly gate set. Given that we will be running this on a simulator, qubit connectivity is not a problem, so we will not need to route the circuit, hence it is safe to use `optimise_pre_routing` and make use of the extra optimisation techniques.

Aer's simulators also reorder the most-/least-significant qubits for their statevector and unitary readout (when generating the circuit, we assumed qubit 0 is the most significant, but Aer takes this to be the least significant). We can fix this by permuting the qubits using `apply_boundary_map`.

In [3]:
from pytket._circuit import optimise_pre_routing
optimise_pre_routing(tk_circ)
tk_circ.apply_boundary_map([1,0])

The final step is to convert our circuit to one of the Qiskit representations and run it on the simulator.

In [4]:
from pytket.qiskit import tk_to_dagcircuit
dag_circ = tk_to_dagcircuit(tk_circ)

from qiskit.converters import dag_to_circuit
qis_circ = dag_to_circuit(dag_circ)

from qiskit import BasicAer, execute
backend = BasicAer.get_backend('unitary_simulator')
job = execute(qis_circ, backend)
matrix = job.result().get_unitary(qis_circ)
print(matrix)

[[ 5.00000000e-01+0.j   5.00000000e-01+0.j   5.00000000e-01+0.j
   5.00000000e-01+0.j ]
 [ 5.17086374e-14+0.5j -5.17641485e-14-0.5j  5.17919041e-14+0.5j
  -5.18474152e-14-0.5j]
 [ 5.14588372e-14-0.5j  5.14033260e-14-0.5j -5.18474152e-14+0.5j
  -5.17919041e-14+0.5j]
 [ 5.18751708e-14+0.5j -5.19306820e-14-0.5j -5.15143483e-14-0.5j
   5.15698595e-14+0.5j]]


Looking at this, we can see our target state, up to floating-point error, in the first column (i.e. it is produced by applying this matrix to the ground state), but we now get the full picture of what this circuit is doing.

As the quantum software landscape continues to grow and develop, the ability to mix-and-match front- and back-ends will only ever become more beneficial. The retargetability and generality of the t|ket> compiler can help to get the best out of what the field has to offer.